In [1]:
include <- function(pkg) {
  if (!suppressMessages(require(pkg, character.only = TRUE)))
    install.packages(pkg, character.only = TRUE)
  suppressMessages(library(pkg, pkg, character.only = TRUE))
}
include("docopt")
include("stringr")
include("alakazam")
include("scoper")
include("dplyr")
source("helpers.r")

In [2]:
doc <- "Usage:
  clones.R [--name <name>] [--threads <threads>] [--binwidth <binwidth>] [--wd <wd>][--suffix <suffix>] <tsv> 

  Options:   
   -w --wd <wd> [default: .]
   -n --name <name> Name of the sample
   -t --threads <threads> Number of threads [type: int] [default: 8]
   -b --binwidth Width of the bin in plotting [type: num] [default: 0.02]
   -s --suffix <suffix> [default: _with_clones].
   -h --help     Show this screen."

In [3]:
debug <- TRUE
if (debug == TRUE) {
  tsv <- file.path("/data", "samples", "AIRR-Seq", "OURS", "S3987Nr1", "S3987Nr1-PBMC_heavy", "changeo_igblast", "S3987Nr1-PBMC_heavy_f_parse-select_with_translation.tsv")  # 'clones', 'changeo_clone',  'our_pbmc_germ-pass.tsv')    
  args <- paste("--suffix _with_clones --name S3987Nr1-PBMC_heavy", tsv)
  print(args)
  values <- docopt(doc, args = args, version = "0.1")
  print(values)
} else {
  values <- docopt(doc, version = "0.1")
}

[1] "--suffix _with_clones --name S3987Nr1-PBMC_heavy /data/samples/AIRR-Seq/OURS/S3987Nr1/S3987Nr1-PBMC_heavy/changeo_igblast/S3987Nr1-PBMC_heavy_f_parse-select_with_translation.tsv"
List of 14
 $ --wd      : chr "."
 $ --name    : chr "S3987Nr1-PBMC_heavy"
 $ --threads : chr "8"
 $ --binwidth: chr "0.02"
 $ --suffix  : chr "_with_clones"
 $ --help    : logi FALSE
 $ <tsv>     : chr "/data/samples/AIRR-Seq/OURS/S3987Nr1/S3987Nr1-PBMC_heavy/changeo_igblast/S3987Nr1-PBMC_heavy_f_parse-select_wit"| __truncated__
 $ wd        : chr "."
 $ name      : chr "S3987Nr1-PBMC_heavy"
 $ threads   : chr "8"
 $ binwidth  : chr "0.02"
 $ suffix    : chr "_with_clones"
 $ help      : logi FALSE
 $ tsv       : chr "/data/samples/AIRR-Seq/OURS/S3987Nr1/S3987Nr1-PBMC_heavy/changeo_igblast/S3987Nr1-PBMC_heavy_f_parse-select_wit"| __truncated__
NULL


In [4]:
tsv <- values$tsv
name <- values$name
binwidth <- as.numeric(values$binwidth)
threads <- as.numeric(values$threads)

### Spectral clone analysis ###

In [6]:
db <- readChangeoDb(tsv)

print(paste("starting spectral analyzis with ", threads, "threads"))

# Clonal assignment using identical nucleotide sequences
results <- scoper::spectralClones(db, "vj", nproc = threads)

writeAnalysisTable(results@vjl_groups, build_filepath(name, "vjl_groups", "tsv"))

print(paste("writing spectral analyzes results", file.path(paste0(name, values$suffix, ".tsv"))))
writeChangeoDb(results@db, file.path(paste0(name, values$suffix, ".tsv")))

print(paste("writing spectral analyzes picture", file.path(paste0(name, values$suffix, ".png"))))
png(file = paste0(name, values$suffix, ".png"), width = 800, height = 600)
if (binwidth > 0) {
    print(paste("binwidth is", binwidth))
  plot(results, binwidth = binwidth)
} else plot(results)
if (debug == TRUE) {
  dev.off()
}

[1] "starting spectral analyzis with  8 threads"


Running defineClonesScoper in bulk mode



[1] "Output is written to S3987Nr1-PBMC_heavy_vjl_groups.tsv"
[1] "writing spectral analyzes results S3987Nr1-PBMC_heavy_with_clones.tsv"
[1] "writing spectral analyzes picture S3987Nr1-PBMC_heavy_with_clones.png"
[1] "binwidth is 0.02"


png 
  2